In [ ]:
################################################
# New style 
################################################
import sys
import os

workdir_ = '/glade/work/juliob'
if ( workdir_ not in sys.path ):
    sys.path.append(workdir_)
    print( f" a path to {workdir_} added in {__name__} ")

from myPythonTools.Utils import utils as uti
from myPythonTools.Utils import numerical_utils as nuti
from myPythonTools.Utils import AveragingUtils as Av
from myPythonTools.Utils import validation_data as Val
from myPythonTools.Utils import PlotUtil as Pu
from myPythonTools.Plotting import LatLonMaps as LL 
from myPythonTools.CASutils import filter_utils as fu 

from PyRegridding.Utils import GridUtils as GrU
from PyRegridding.Utils import MakePressures as MkP
from PyRegridding.Drivers import RegridField as RgF


# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Some other useful packages 
import importlib
import copy
import time
import cftime

importlib.reload( uti )
importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
#importlib.reload(vAB)
importlib.reload(MkP)
importlib.reload(RgF)


In [ ]:
print( sys.path )
############################
# Initialize dict for regridding
############################
RgObs={}


In [ ]:
### Make a list of files
import yaml
#from box import Box


with open('configure_ana_plots.yaml', 'r') as file:
    cfg = yaml.safe_load(file)


######################################################
season = cfg['plot']['season']
valData  = cfg['validation']['data']
valYears = cfg['validation']['years']

x='oldCTL'#'alpha04ax010'
x='ne30x5' #x2_QxQsst'
x='ne240x2_QxQsst' #_smth'
x='CAM7_ne120_FMT' #_smth'
x='CAM7_ne30_FMT' #_smth'

xl= ['oldCTL', 'c153_ne120_x2' ,'ne240x2', 'newCTL' ] #'ne240x2_QxQsst']
Al =[]
for x in xl:
    exp, subd, Src, Hkey, Dst, useri = cfg[x]['name'] , cfg[x]['subdir'] , cfg[x]['SrcGrid'] , cfg[x]['Hkey'] , cfg[x]['DstGrid'] , cfg[x]['user'] 
    ymdPat = cfg[x]['ymdPat']
    print( exp, subd, Src, Hkey, Dst, useri )
    print( ymdPat , season )
    print( valData , valYears )
    
    A = uti.MakeDict4Exp( exp=exp , user=useri, subd=subd , 
                     hsPat='cam.h0a' , ymdPat=ymdPat,verbose=True, open_dataset=True )
    A['Src']=Src
    A['Hkey']=Hkey
    Al.append( A )


In [ ]:
#Al[0]

In [ ]:
########################################
# Make seasonal means
########################################
season='son'

for A in Al:
    print( f'{A.exp} Years {len(A.X.time) /12. }')
    
    UUa_s,years_a,months_a = Av.Seasonal( ds=A.X, season=season , fld='U',return_time=True)
    TTa_s,years_a,months_a = Av.Seasonal( ds=A.X, season=season , fld='T',return_time=True)
    A['UU_s']=UUa_s
    A['TT_s']=TTa_s
    A['years']=years_a
    A['months']=months_a


In [ ]:
#Al[2]['UUa_s'].shape

In [ ]:

########################################
# Make zonal means
########################################
method='C2'

for A in Al:
    Hkey = A['Hkey']
    Src  = A['Src']
    UUa_s = A['UU_s']
    TTa_s = A['TT_s']
    
    if (Hkey == 'yx' ):
        ########################
        # Option 1 from lat-lon regirrdded data
        ########################

        lon_a=A.X.lon.values
        lat_a=A.X.lat.values
        lev_a=A.X.lev.values
        zlev_a=-7.0*np.log( lev_a/1_000. )
        # Zonal Av
        UUa_sz=np.average( UUa_s , axis=2 )
        TTa_sz=np.average( TTa_s , axis=2 )
    
    elif (Hkey=='c'):
        ########################
        # Option 2 from unstructured SE data
        ########################
        
        Dst = 'fv1x1'
        DstInfo = GrU.gridInfo(Dst) #,Vgrid=DstVgrid)
        lat_a,lon_a = GrU.latlon( scrip= DstInfo['scrip'], Hkey=DstInfo['Hkey'] )
        lev_a=A.X.lev.values
        zlev_a=-7.0*np.log( lev_a/1_000. )
    
        RegridObj = GrU.regrid_object_lib(RgOb=RgObs, src=Src, dst=Dst)
        UUa_s_xy=RgF.Horz(xfld_Src=UUa_s , Src=Src, Dst=Dst , RegridObj_In= RegridObj )
        TTa_s_xy=RgF.Horz(xfld_Src=TTa_s , Src=Src, Dst=Dst , RegridObj_In= RegridObj )
        
        # Zonal Av
        UUa_sz=np.average( UUa_s_xy , axis=2 )
        TTa_sz=np.average( TTa_s_xy , axis=2 )

    A['UU_sz'] = UUa_sz
    A['TT_sz'] = TTa_sz
    A['lonx'] = lon_a
    A['latx'] = lat_a
    A['DateRange'] = f"{(A.X.time[0].values.item() ).strftime('%Y-%m-%d')}_{(A.X.time[-1].values.item() ).strftime('%Y-%m-%d')}"

In [ ]:
Al[2].UU_sz.shape

In [ ]:
####
# Validation
####
Validation_on_control_mgrid = False

if (Validation_on_control_mgrid == True):
    PSc_s = Av.Seasonal( ds=Dc, season=season , fld='PS')
    mgrid={'ps':PSc_s, 'hyam':hyam_c, 'hybm':hybm_c, 'hyai':hyai_c, 'hybi':hybi_c, 'hgrid':'fv0.9x1.25' }
else:
    mgrid=False


valYears='*'
Uval = Val.data(fld='U',season=season,mgrid=mgrid,zlev=True , Years=valYears ) #, Years='1996' ) #, Years='2000' )
UUv_s , zlev_v,lat_v,lon_v, years_Uv = Uval['aa'], Uval['lev'], Uval['lat'], Uval['lon'], Uval['years']
UUv_sz=np.average( UUv_s , axis=2 )

Tval = Val.data(fld='T',season=season,mgrid=mgrid,zlev=True , Years=valYears ) #, Years='1996' ) #, Years='2000' )
TTv_s , zlev_v,lat_v,lon_v, years_Tv = Tval['aa'], Tval['lev'], Tval['lat'], Tval['lon'], Tval['years']
TTv_sz=np.average( TTv_s , axis=2 )



In [ ]:
print(len(Al))

In [ ]:


#nxplo,nyplo=4,1
#fig = plt.figure(figsize=(24, 5))

numx=len( Al )
nxplo,nyplo=numx+1,2
fig = plt.figure(figsize=(8*nxplo, 18))

ulev=np.linspace( -60,140,num=21) 
tlev=np.linspace( 180,300,num=21) 
dlev=np.linspace( -20,20,num=21) 
cmap='gist_ncar'

titlesize=18

plotTop=85

ix=0

for A in Al:
    
    UUa_sz = A['UU_sz'] 
    TTa_sz = A['TT_sz'] 
    lon_a = A['lonx']
    lat_a = A['latx']
    years_a = A['years']
    
    # Create 2D arrays
    Lat_a, Zlev_a = np.meshgrid(lat_a, zlev_a)
    
    # Adding the hatching for the sponge layer
    sponge_layer = np.zeros_like(UUa_sz)
    sponge_layer[(Zlev_a >= 62) & (Zlev_a <= 85)] = 1
    
    title = f"<{A.expdesc}> \n {season.upper()} {years_a[0]}-{years_a[-1]}"
    n=ix+1
    Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo, ypan=0.8 ) 
    ax1 = fig.add_axes( Axes1 )
    co1=ax1.contourf(lat_a,zlev_a,UUa_sz ,levels=ulev, cmap=cmap )
    co2=ax1.contour(lat_a,zlev_a,UUa_sz ,levels=ulev, colors='black')
    #ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
    ax1.set_title( title ,fontsize=titlesize )
    ax1.set_ylim(0,plotTop)
    cb=plt.colorbar(co1)
    ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
        fontsize=20, fontweight='bold', va='top')
    
    # Sponge layer hatching
    ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)
    
    
    n=ix+nxplo+1
    Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo, ypan=.8 ) 
    ax1 = fig.add_axes( Axes1 )
    co1=ax1.contourf(lat_a,zlev_a, TTa_sz ,levels=tlev, cmap=cmap )
    co2=ax1.contour(lat_a,zlev_a, TTa_sz ,levels=tlev, colors='black')
    #ax1.set_title(f"Validation <{exp_Val}> {season.upper()}  " )
    ax1.set_title( title  ,fontsize=titlesize )
    ax1.set_ylim(0,plotTop)
    cb=plt.colorbar(co1)
    ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
        fontsize=20, fontweight='bold', va='top')
    # Sponge layer hatching
    ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)

    ix=ix+1

for ix in np.arange( start=numx,stop=numx+1 ):

    title = f"<{valData}> {season.upper()} {years_Uv} " 
    n=ix+1
    Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo, ypan=0.8 ) 
    ax1 = fig.add_axes( Axes1 )
    co1=ax1.contourf(lat_v,zlev_v,UUv_sz ,levels=ulev, cmap=cmap )
    co2=ax1.contour(lat_v,zlev_v,UUv_sz ,levels=ulev, colors='black')
    ax1.set_title( title ,fontsize=titlesize )
    ax1.set_ylim(0,plotTop)
    cb=plt.colorbar(co1)
    ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
        fontsize=20, fontweight='bold', va='top')

    n=ix+nxplo+1
    Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo, ypan=.8 ) 
    ax1 = fig.add_axes( Axes1 )
    co1=ax1.contourf(lat_v,zlev_v, TTv_sz ,levels=tlev, cmap=cmap )
    co2=ax1.contour(lat_v,zlev_v, TTv_sz ,levels=tlev, colors='black')
    ax1.set_title( title  ,fontsize=titlesize )
    ax1.set_ylim(0,plotTop)
    cb=plt.colorbar(co1)
    ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
        fontsize=20, fontweight='bold', va='top')




FigName = f"AGU_2024_poster_zonalUT_{season}.png" #f"{A.exp}_{season.upper()}_{years_a[0]}-{years_a[-1]}_Dst{Dst}_zUT_climo.png"
print( FigName) 
Pu.save_link(png_filename = FigName )



In [ ]:

Al[0]['expdesc'] = 'CAM7_dev0: 1 degree '
Al[1]['expdesc'] = 'CAM7_dev0: 0.25 degree'
Al[2]['expdesc'] = 'CAM7_dev0: 0.125 degree'
Al[3]['expdesc'] = 'CAM7_dev1: 1 degree w/ PBL-frontal GW'
